# <div align="center">Regression</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


The 3D regression module consists of two fully-connected layers with 1024 neurons each with a dropout layer in between, followed by a final layer of 85D neurons. We use T = 3 iterations for all of our experiments.

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers.initializers import variance_scaling_initializer

In [2]:
def Encoder_resnet(x, is_training=True, weight_decay=0.001, reuse=False):
    """
    Resnet v2-50
    Assumes input is [batch, height_in, width_in, channels]!!
    Input:
    - x: N x H x W x 3
    - weight_decay: float
    - reuse: bool->True if test

    Outputs:
    - cam: N x 3
    - Pose vector: N x 72
    - Shape vector: N x 10
    - variables: tf variables
    """
    from tensorflow.contrib.slim.python.slim.nets import resnet_v2
    # with tf.name_scope("Encoder_resnet", [x]):
    with tf.name_scope("Encoder_resnet"):
        with slim.arg_scope(resnet_v2.resnet_arg_scope(weight_decay=weight_decay)):
            net, end_points = resnet_v2.resnet_v2_50(
                x,
                num_classes=None,
                is_training=is_training,
                reuse=reuse,
                scope='resnet_v2_50')
            net = tf.squeeze(net, axis=[1, 2])
    variables = tf.contrib.framework.get_variables('resnet_v2_50')
    return net, variables

In [3]:
def Encoder_fc3_dropout(x,
                        num_output=85,
                        is_training=True,
                        reuse=False,
                        name="3D_module"):
    """
    3D inference module. 3 MLP layers (last is the output)
    With dropout  on first 2.
    Input:
    - x: N x [|img_feat|, |3D_param|]
    - reuse: bool

    Outputs:
    - 3D params: N x num_output
      if orthogonal: 
           either 85: (3 + 24*3 + 10) or 109 (3 + 24*4 + 10) for factored axis-angle representation
      if perspective:
          86: (f, tx, ty, tz) + 24*3 + 10, or 110 for factored axis-angle.
    - variables: tf variables
    """
    if reuse:
        print('Reuse is on!')
    with tf.variable_scope(name, reuse=reuse) as scope:
        net = slim.fully_connected(x, 1024, scope='fc1')
        net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout1')
        net = slim.fully_connected(net, 1024, scope='fc2')
        net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout2')
        small_xavier = variance_scaling_initializer(
            factor=.01, mode='FAN_AVG', uniform=True)
        net = slim.fully_connected(
            net,
            num_output,
            activation_fn=None,
            weights_initializer=small_xavier,
            scope='fc3')

    variables = tf.contrib.framework.get_variables(scope)
    return net, variables

In [4]:
def get_encoder_fn_separate(model_type):
    """
    Retrieves diff encoder fn for image and 3D
    """
    encoder_fn = None
    threed_fn = None
    if 'resnet' in model_type:
        encoder_fn = Encoder_resnet
    else:
        print('Unknown encoder %s!' % model_type)
        exit(1)

    if 'fc3_dropout' in model_type:
        threed_fn = Encoder_fc3_dropout

    if encoder_fn is None or threed_fn is None:
        print('Dont know what encoder to use for %s' % model_type)
        import ipdb
        ipdb.set_trace()

    return encoder_fn, threed_fn

In [5]:
img_enc_fn, threed_enc_fn = get_encoder_fn_separate('resnet_fc3_dropout')

In [6]:
import skimage.io as io
import numpy as np
from src.util import image as img_util
from matplotlib import pyplot as plt

In [7]:
def preprocess_image(img_path, json_path=None):
    img = io.imread(img_path)
    if img.shape[2] == 4:
        img = img[:, :, :3]

    if json_path is None:
        if np.max(img.shape[:2]) != 224:
            print('Resizing so the max image size is %d..' % 224)
            scale = (float(224) / np.max(img.shape[:2]))
        else:
            scale = 1.
        center = np.round(np.array(img.shape[:2]) / 2).astype(int)
        # image center in (x,y)
        center = center[::-1]
    else:
        scale, center = op_util.get_bbox(json_path)

    crop, proc_param = img_util.scale_and_crop(img, scale, center, 224)

    # Normalize image to [-1, 1]
    crop = 2 * ((crop / 255.) - 0.5)

    return crop, proc_param, img

In [8]:
img_path = 'data/im1963.jpg'
json_path = None
input_img, proc_param, img = preprocess_image(img_path, json_path)
input_img = np.expand_dims(input_img, 0)

Resizing so the max image size is 224..


In [9]:
input_img = np.float32(input_img)

In [10]:
# Extract image features.
img_feat, E_var = img_enc_fn(input_img, is_training=False, reuse=False)

In [11]:
# flags.DEFINE_integer('num_stage', 3, '# of times to iterate regressor')
num_stage = 3

In [12]:
import os.path as osp
import os
import sys
curr_path = osp.dirname(os.getcwd())
model_dir = osp.join(curr_path, '..', 'models')

# SMPL_MODEL_PATH = osp.join(model_dir, 'neutral_smpl_with_cocoplus_reg.pkl')
# SMPL_FACE_PATH = osp.join(curr_path, '../src/tf_smpl', 'smpl_faces.npy')

# smpl_model_path = SMPL_MODEL_PATH
# smpl_face_path = SMPL_FACE_PATH

smpl_model_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\models\neutral_smpl_with_cocoplus_reg.pkl'
smpl_face_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\smpl_faces.npy'

In [13]:
import deepdish as dd
from os.path import join, dirname
def load_mean_param():
    mean = np.zeros((1, 85))
    # Initialize scale at 0.9
    mean[0, 0] = 0.9
    mean_path = join(
        dirname(smpl_model_path), 'neutral_smpl_mean_params.h5')
    mean_vals = dd.io.load(mean_path)

    mean_pose = mean_vals['pose']
    # Ignore the global rotation.
    mean_pose[:3] = 0.
    mean_shape = mean_vals['shape']

    # This initializes the global pose to be up-right when projected
    mean_pose[0] = np.pi

    mean[0, 3:] = np.hstack((mean_pose, mean_shape))
    mean = tf.constant(mean, tf.float32)
    mean_var = tf.Variable(
        mean, name="mean_param", dtype=tf.float32, trainable=True)
    E_var.append(mean_var)
    init_mean = tf.tile(mean_var, [1, 1])
    return init_mean

In [14]:
from src.tf_smpl.batch_lbs import batch_rodrigues
from src.tf_smpl.batch_smpl import SMPL
from src.tf_smpl.projection import batch_orth_proj_idrot

In [15]:
smpl = SMPL(smpl_model_path)

In [24]:
img_feat

<tf.Tensor 'Encoder_resnet/Squeeze:0' shape=(1, 2048) dtype=float32>

In [23]:
theta_prev

<tf.Tensor 'Tile:0' shape=(1, 85) dtype=float32>

In [16]:
num_cam=3
num_theta=72
loss_kps = []
# if self.use_3d_label:
#     loss_3d_joints, loss_3d_params = [], []
# For discriminator
fake_rotations, fake_shapes = [], []
# Start loop
# 85D
theta_prev = load_mean_param()

# For visualizations
all_verts = []
all_pred_kps = []
all_pred_cams = []
all_delta_thetas = []
all_theta_prev = []

num_stage = 1

# Main IEF loop
for i in np.arange(num_stage):
    print('Iteration %d' % i)
    # ---- Compute outputs
    state = tf.concat([img_feat, theta_prev], 1)

    if i == 0:
        delta_theta, threeD_var = threed_enc_fn(
            state, num_output=85, reuse=False)
        E_var.extend(threeD_var)
    else:
        delta_theta, _ = threed_enc_fn(
            state, num_output=85, reuse=True)

    # Compute new theta
    theta_here = theta_prev + delta_theta
    # cam = N x 3, pose N x self.num_theta, shape: N x 10
    cams = theta_here[:, : num_cam]
    poses = theta_here[:, num_cam:(num_cam + num_theta)]
    shapes = theta_here[:, (num_cam + num_theta):]
    
    # Rs_wglobal is Nx24x3x3 rotation matrices of poses
    verts, Js, pred_Rs = smpl(shapes, poses, get_skin=True)
    pred_kp = batch_orth_proj_idrot(
        Js, cams, name='proj2d_stage%d' % i)
#     # --- Compute losses:
#     loss_kps.append(self.e_loss_weight * self.keypoint_loss(
#         self.kp_loader, pred_kp))
#     pred_Rs = tf.reshape(pred_Rs, [-1, 24, 9])
#     if self.use_3d_label:
#         loss_poseshape, loss_joints = self.get_3d_loss(
#             pred_Rs, shapes, Js)
#         loss_3d_params.append(loss_poseshape)
#         loss_3d_joints.append(loss_joints)

#     # Save pred_rotations for Discriminator
#     fake_rotations.append(pred_Rs[:, 1:, :])
#     fake_shapes.append(shapes)

#     # Save things for visualiations:
#     self.all_verts.append(tf.gather(verts, self.show_these))
#     self.all_pred_kps.append(tf.gather(pred_kp, self.show_these))
#     self.all_pred_cams.append(tf.gather(cams, self.show_these))

    # Finally update to end iteration.
    theta_prev = theta_here


Iteration 0


W0216 15:30:16.864979  4980 deprecation.py:323] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\batch_lbs.py:55: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [22]:
verts

<tf.Tensor 'smpl_main/strided_slice_3:0' shape=(1, 6890, 3) dtype=float32>

In [25]:
pred_kp

<tf.Tensor 'proj2d_stage0/Reshape_1:0' shape=(1, 19, 2) dtype=float32>

In [26]:
pred_Rs

<tf.Tensor 'smpl_main/Reshape_2:0' shape=(1, 24, 3, 3) dtype=float32>